# core

> Python package that helps collect outputs of statistical analyses into tables and export them to LaTex and pdf (similar the ``estout`` and ``esttab`` commands in Stata).

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import importlib
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels import PanelOLS

from estout.utils import *

In [ ]:
np.random.seed(0)
df = pd.DataFrame(np.random.rand(9,2), columns=['y','x'])
df['firmid'] = [1]*3 + [2]*3 + [3]*3
df['time'] = [1,2,3]*3
df['constant'] = 1
df = df.set_index(['firmid','time'])
df

y         x  constant
firmid time                              
1      1     0.548814  0.715189         1
       2     0.602763  0.544883         1
       3     0.423655  0.645894         1
2      1     0.437587  0.891773         1
       2     0.963663  0.383442         1
       3     0.791725  0.528895         1
3      1     0.568045  0.925597         1
       2     0.071036  0.087129         1
       3     0.020218  0.832620         1

In [ ]:
sm1 = sm.OLS(df['y'], df[['constant','x']]).fit()
sm2 = sm.OLS(df['y'], df[['constant','x']]).fit().get_robustcov_results(cov_type='HAC', maxlags=2)
lmres = PanelOLS(df['y'],  df[['constant','x']], entity_effects=True
                 ).fit(cov_type='clustered', cluster_entity=True)

In [ ]:
#| export
def extract_stat(res, package: str=None, stat: str=None): 
    results_package = importlib.import_module(f"estout.{package}_results")
    return rgetattr(results_package,stat)(res)

In [ ]:
extract_stat(lmres, package='linearmodels', stat='tstats')

constant    2.014245
x          -0.014231
Name: tstat, dtype: float64

In [ ]:
extract_stat(sm1, package='statsmodels', stat='tstats')

constant    1.762032
x          -0.044981
dtype: float64

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()